In [ ]:
!pip install datasets
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes
!pip install -U transformers accelerate


!pip install -U bitsandbytes # Update bitsandbytes to the latest version
import os
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from google.colab import files  # For file upload and download in Colab

# ... (rest of your code) ...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
   

In [ ]:
import os
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from google.colab import files  # For file upload and download in Colab

# Set the current directory for Google Colab
current_dir = "/content"  # Default directory in Colab

# Directly define model and tokenizer settings
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"  # Change as needed
embedding_model_name_reader = "sentence-transformers/all-mpnet-base-v2"  # Example, modify as needed
chunk_size_reader = 512
chunk_overlap_reader = 50

# Load the 4-bit quantized causal LLM (This can be adjusted for your specific model)
from transformers import BitsAndBytesConfig
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Quantization config for 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",  # Or torch.float16 if bfloat16 is not supported
)

# Load the model with quantization
generative_model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)
tokenizer_reader = AutoTokenizer.from_pretrained(READER_MODEL_NAME)


# Define the text generation pipeline for later use
READER_LLM = pipeline(
    model=generative_model,
    tokenizer=tokenizer_reader,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

# Use file uploader to browse and select the dataset file
uploaded = files.upload()  # This will trigger file selection in the Colab UI
scraped_content_path = next(iter(uploaded))  # Get the filename of the uploaded file

# Load the actual model for fine-tuning (without the pipeline)
from transformers import AutoModelForCausalLM

# Load the model (not from pipeline this time)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME)

# Load the dataset from the uploaded file (JSON format)
dataset = load_dataset("json", data_files=scraped_content_path, split="train")

# Tokenization and preprocessing function
def preprocess_function(examples):
    # Tokenize both the instruction and response for training
    return tokenizer_reader(examples["instruction"], examples["response"], truncation=True, padding="max_length")

# Tokenize the dataset using map function for batch processing
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=os.path.join(current_dir, "fine_tuned_model"),  # Save the model in the fine-tuned folder
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_dir=os.path.join(current_dir, "logs"),
    logging_steps=500,
    learning_rate=5e-5,
    fp16=True,  # Use mixed precision training for faster training
)

# Create Trainer instance with the model and training arguments
trainer = Trainer(
    model=model,  # Pass the model object here
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start model training
trainer.train()

# Save the fine-tuned model and tokenizer
model_dir = os.path.join(current_dir, "model")  # Save the fine-tuned model in the 'model' folder
model.save_pretrained(model_dir)
tokenizer_reader.save_pretrained(model_dir)

# Notify about the saved model directory
print(f"Model saved to {model_dir}")

# Now download the saved model files
!zip -r /content/fine_tuned_model.zip {model_dir}  # Create a zip file for easy download
files.download("/content/fine_tuned_model.zip")  # Initiate download



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]